In [ ]:
%matplotlib inline
import os
import numpy as np
import json
filepath='D:/measuring'
import scipy.signal
c = 2.997e8
os.chdir(filepath)
from analysis.lib.fitting import fit, common; reload(common)
from analysis.lib.tools import plot


In [ ]:
import analysis.scripts.cavity.oscilloscope_analysis_linewidth as oscillo_ana

In [ ]:
#functions for analysis of linewidths from several files, 
#taking into account the up down modulation

def get_up_down_lws(fitted_lws,first_lw_direction):
    even_lws=fitted_lws[::2]
    odd_lws = fitted_lws[1::2]
    if len(even_lws)>len(odd_lws):
        even_lws=even_lws[:-1]
    updown_mean_lws=np.add(even_lws,odd_lws)/2        
    if first_lw_direction=='up':
        lws_up = even_lws
        lws_down  = odd_lws
    elif first_lw_direction=='down':
        lws_up = odd_lws
        lws_down  = even_lws
    else:
        print 'specify a valid modulation direction for the first linewidth'
        return
    return updown_mean_lws,lws_up,lws_down

def add_to_analysis_dict(analysis,lws_up,lws_down,updown_mean_lws):
    analysis['lws_up']=lws_up
    analysis['lws_down']=lws_down
    analysis['updown_mean_lws']=updown_mean_lws
    analysis['avg_up']=np.average(lws_up)
    analysis['u_avg_up']=np.std(lws_up)
    analysis['avg_down']=np.average(lws_down)
    analysis['u_avg_down']=np.std(lws_down)
    analysis['avg_updown_mean']=np.average(updown_mean_lws)
    analysis['u_avg_updown_mean']=np.std(updown_mean_lws)
    return analysis

def print_results(analysis):
    print 'avg_up', analysis['avg_up'],analysis['u_avg_up']
    print 'avg_down', analysis['avg_down'],analysis['u_avg_down']
    print 'avg_updown_mean', analysis['avg_updown_mean'],analysis['u_avg_updown_mean']

#functions for saving in json
def default(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError('Not serializable')

def save_to_json_file(indir,filename,analysis_dict):
    f = open(os.path.join(indir,filename+'.json'), 'w')
    json.dump(analysis_dict,f,default=default)
    f.close()

def save_to_txt_file(indir,filename,nparray):
    f = open(os.path.join(indir,filename+'.txt'), 'w')
    np.savetxt(f,nparray)
    f.close()
  

In [ ]:

# Parameters for

#g_x02 = 0.02
# g_gamma2 = 0.002

nr_datafiles = 9

linewidths = np.zeros((nr_datafiles,2))

reload(oscillo_ana)

for i in np.arange(nr_datafiles):
    fixed =[]#[0,1,2,5]
    
    g_a1 = 0.002
    g_A1 = 0.00004
    g_x01 = 0.002
    g_gamma1 = 0.0004
    g_dx = 0.0015
    g_A2 = 0.00002
    
    if i == 2:
        g_gamma1 = 0.0035
    indir='D:\measuring\data/20160430\OFF_diamond'#'D:\measuring\data/20160430\OFF_diamond'
    filename='OFFD00'+str(i)
    linewidth, u_linewidth = oscillo_ana.plot_and_fit(indir, filename,
                                                      g_a1, g_A1, g_x01, g_gamma1, g_dx, g_A2,fixed=fixed)
    print linewidth, u_linewidth
    linewidths[i,0]= linewidth
    linewidths[i,1] = u_linewidth
print linewidths

f = open(os.path.join(indir,'linewidth_data.txt'), 'w')
np.savetxt(f,linewidths)
f.close()

In [ ]:
nu=470.e12
Q = nu/(linewidths[1])
print Q
nu_FSR=c/(2*cavity_lengths[1])
F=Q*nu_FSR/nu
print F
F=nu_FSR/linewidths[1]
print F
print nu_FSR


In [ ]:
def get_avg_values_from_json(V,Lnr,day_dir,tag=''):
    V_str = '%1.3f'%V
    string_name = '%sL%d_V%sp%s'%(tag,Lnr,V_str[0],V_str[2:])
    #print string_name
    datapath = os.path.join(day_dir,string_name)
    f = open(os.path.join(datapath,'analysis.json'), 'r')
    analysis = json.load(f)
    f.close()
    avg_values = np.array([[analysis['avg_up'], analysis['u_avg_up'],\
        analysis['avg_down'],analysis['u_avg_down'],\
        analysis['avg_updown_mean'],analysis['u_avg_updown_mean']]])
    return datapath,avg_values

def get_avg_array_for_Vs(Vs,Lnr,day_dir,tag=''):
    for i,V in enumerate(Vs):
        datapath,avg_values=get_avg_values_from_json(V,Lnr,day_dir,tag=tag)
        if i==0:
            datapath,avg_array=get_avg_values_from_json(V,Lnr,day_dir,tag=tag)
        else:
            avg_array=np.append(avg_array,avg_values,axis=0)
    return datapath,avg_array


In [ ]:
def plot_updown_lws(avgs,Ns,ax=None):
    if ax==None:
        fig,ax = plt.subplots()
    ax.errorbar(Ns,avgs[:,0],yerr=avgs[:,1],fmt='ro',label='upgoing slope')
    ax.errorbar(Ns,avgs[:,2],yerr=avgs[:,3],fmt='go',label='downgoing slope')
    ax.errorbar(Ns,avgs[:,4],yerr=avgs[:,5],fmt='bo',label='up and down mean')
    ax.set_xlabel('cavity length (lambda/2)')
    ax.set_ylabel('linewidth (GHz)')
    ax.legend()
    return(ax)

def plot_mean_lws(avgs,Ns,ax=None,style='bo',label='mean'):
    if ax==None:
        fig,ax = plt.subplots()
    ax.errorbar(Ns,avgs[:,4],yerr=avgs[:,5],fmt=style,label=label)
    ax.set_xlabel('cavity length (lambda/2)')
    ax.set_ylabel('linewidth (GHz)')
    ax.legend()
    return(ax)


def plot_finesse_vs_length(avgs,Ns,lambda_c=636.6e-9):
    Finesses = c/(np.multiply(Ns,lambda_c))/np.multiply(avgs[:,4],1.e9)
    u_Finesses = c/(np.multiply(Ns,lambda_c))/np.multiply((avgs[:,4]**2),1.e9)*avgs[:,5]
    fig,ax = plt.subplots()
    print Finesses,u_Finesses
    ax.errorbar(Ns,Finesses,yerr=u_Finesses,fmt='bo',label='mean Finesse')
    ax.set_xlabel('cavity length (lambda/2)')
    ax.set_ylabel('Finess')
    ax.legend()
    return(ax)

    

def fit_lws(linewidths,u_linewidths,Ns,g_Finesse,datapath,ax=None,lambda_c=636.6e-9):
    #cavity_lengths =  Ns#636.6e-9*Ns/2 # in um
    g_a = 0#0.1*1.e9
    g_b = (c)/(2*g_Finesse)*1.e-9*2/lambda_c #*1.e9
    print g_b
    print Ns
    print linewidths
    p0, fitfunc, fitfunc_str = common.fit_inverse(g_a, g_b)
    fit_result = fit.fit1d(Ns,linewidths, None, 
                           p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=[0])
    if ax == None:
        fig,ax = plt.subplots(figsize=(8,4))
        ax.errorbar(Ns,linewidths,yerr=u_linewidths,fmt = 'bo')
    x = np.linspace(min(Ns),max(Ns),10*len(Ns))
    plot.plot_fit1d(fit_result,x, ax=ax, label='Fit',show_guess=True, 
                    plot_data=False,color='red', print_info= False)
    b_fit=fit_result['params_dict']['b']#cav length in um, linewidth in GHz
    u_b_fit = fit_result['error_dict']['b']
    print b_fit
    Finesse = c/(2*b_fit)*1.e-9*2/lambda_c
    u_Finesse = c/(2*b_fit**2)*1.e-9*2/lambda_c*u_b_fit
    ax.set_title(datapath)
    ax.legend(title='Finesse = %d $\pm$ %d'%(Finesse,u_Finesse))
    plt.savefig(os.path.join(datapath,'plot_lw_vs_L.png'))
    plt.show()
    plt.close()
    return Finesse

In [ ]:
# off diamond values

day_dir_20160818 = 'D:\measuring\data/20160818'
V0s = np.array([1.129,2.473,3.482])
V2s = np.array([6.083,7.258,8.269,9.246])
day_dir_20160819 = 'D:\measuring\data/20160819'
V3s = np.array([1.402,2.646,3.781,4.846,5.864,6.890])
N0s = np.array([21,22,23])#possibly the N0s should be 21,22,23->not 100%clear. -> leave out?
N2s = np.array([13,14,15,16])
N3s = np.array([17,18,19,20,21,22])

datapath,avgs0=get_avg_array_for_Vs(V0s,0,day_dir_20160818)
datapath,avgs2=get_avg_array_for_Vs(V2s,2,day_dir_20160818)
datapath,avgs3=get_avg_array_for_Vs(V3s,3,day_dir_20160819)

avgs = np.append(np.append(avgs0,avgs2,axis=0),avgs3,axis=0)
Ns = np.append(np.append(N0s,N2s),N3s)

linewidths = avgs[:,4]
u_linewidths = avgs[:,5]
ax = plot_updown_lws(avgs,Ns)
Finesse=fit_lws(linewidths,u_linewidths,Ns,g_Finesse=30000,datapath=datapath, ax=ax)
plot_finesse_vs_length(avgs,Ns)
print Finesse



In [ ]:
# ON DIAMOND LINEWIDTH< LENGHTS< POS0
day_dir_20160819 = 'D:\measuring\data/20160819'
V0s=np.array([2.935,4.088,5.183,6.182])
V1s=np.array([2.139,3.587,4.753,5.836,6.891,7.928,8.909,9.958])
V2s=np.array([6.957,8.738,9.866])

N0s = np.array([54,55,56,57])
N1s = np.array([45,46,47,48,49,50,51,52])#np.array([44,45,46,47,48,49,50,51])
N2s = np.array([52,53,54])#np.array([51,52,53])
print N0s
print N1s 
print N2s


datapath,avgs0=get_avg_array_for_Vs(V0s,0,day_dir_20160819,tag='OND_')
datapath,avgs1=get_avg_array_for_Vs(V1s,1,day_dir_20160819,tag='OND_')
datapath,avgs2=get_avg_array_for_Vs(V2s,2,day_dir_20160819,tag='OND_')

avgs = np.append(np.append(avgs1,avgs2,axis=0),avgs0,axis=0)
Ns = np.append(np.append(N1s,N2s),N0s)

#beunen begint hier
avgsN52=[(avgs[7]+avgs[8])/2.]
avgsN54=[(avgs[10]+avgs[11])/2.]
avgspos0 = np.append(np.append(np.append(np.append(avgs[0:7],avgsN52,axis=0),[avgs[9]],axis=0),avgsN54,axis=0),avgs[12:],axis=0)
print avgspos0
N52=[(Ns[7]+Ns[8])/2.]
N54=[(Ns[10]+Ns[11])/2.]
Nspos0 = np.append(np.append(np.append(np.append(Ns[0:7],N52),Ns[9]),N54),Ns[12:])
print Nspos0
#beunenn stopt hier

ax = plot_updown_lws(avgspos0,Nspos0)

ax.set_title(datapath)
plt.savefig(os.path.join(datapath,'plot_lw_vs_L.png'))
plt.show()

plot_finesse_vs_length(avgs,Ns)


In [ ]:
# ON DIAMOND LINEWIDTH< LENGHTS< POS1 L2
day_dir_20160822 = 'D:\measuring\data/20160822'
V0s=np.array([1.546,3.019,4.237,5.348,6.404,7.473,8.515,9.551])
V3s=np.array([1.373,2.859,4.062,5.152,6.218,7.224,8.198,9.268])
V4s=np.array([0.951,2.390,3.646,4.783,5.843,6.934,7.975,8.977])
V5s=np.array([1.359,3.383,4.565,5.620,6.695,7.727,8.734,9.832])

N0s = np.arange(55,63,1)
N3s = np.arange(49,57,1)
N4s = np.arange(44,52,1)
N5s = np.arange(40,48,1)
print V3s
print N3s
print N4s
# 
datapath,avgs0=get_avg_array_for_Vs(V0s,1,day_dir_20160822,tag='OND_POS2_')
datapath,avgs3=get_avg_array_for_Vs(V3s,3,day_dir_20160822,tag='OND_POS2_')
datapath,avgs4=get_avg_array_for_Vs(V4s,4,day_dir_20160822,tag='OND_POS2_')
datapath,avgs5=get_avg_array_for_Vs(V5s,5,day_dir_20160822,tag='OND_POS2_')


avgs=np.append(np.append(np.append(avgs0,avgs3,axis=0),avgs4,axis=0),avgs5,axis=0)
Ns=np.append(np.append(np.append(N0s,N3s),N4s),N5s)
# avgs = np.append(np.append(avgs0,avgs1,axis=0),avgs2,axis=0)
# Ns = np.append(np.append(N0s,N1s),N2s)

ax = plot_updown_lws(avgs,Ns)
ax.set_title(datapath)
plt.savefig(os.path.join(datapath,'plot_lw_vs_L.png'))
plt.show()

plot_finesse_vs_length(avgs,Ns)

ax = plot_mean_lws(avgs,Ns, label='mean pos 1')
ax = plot_mean_lws(avgspos0,Nspos0, ax=ax, style ='go',label='mean pos 0')
ax.set_title(datapath)
plt.savefig(os.path.join(datapath,'plot_lw_vs_L_2positions.png'))
plt.show()

In [ ]:
# ON DIAMOND LINEWIDTH< LENGHTS< POS0
#initial analayysis (manual)
linewidths1 = np.array([1.65,1.46, 1.62,1.70,1.67, 1.49,1.24,1.18])
linewidths2 = np.array([0.809,0.798,0.799,0.792])
linewidths3 = np.array([0.93,1.03,1.19])
u_linewidths1 = np.array([0.06,0.04,0.04,0.04,0.04,0.04,0.04,0.04])
u_linewidths2 = np.array([0.03,0.033,0.031,0.034])
u_linewidths3 = np.array([0.04,0.04,0.04])

linewidths=np.append(np.append(linewidths1 ,linewidths2),linewidths3)
u_linewidths=np.append(np.append(u_linewidths1 ,u_linewidths2),u_linewidths3)


In [ ]:
# OFF DIAMOND LINEWIDTH< LENGHTS< 

linewidths = np.array([1.26,1.17,1.09,1.04,0.97,0.92,0.88,0.83,0.79,0.77])*1.e9
u_linewidths = np.array([0.04,0.05,0.05,0.05,0.03,0.04,0.03,0.03,0.04,0.03])*1.e9
N = np.array([13,14,15,16,17,18,19,20,21,22])
cavity_lengths =  636.6*1.e-9*N/2
print linewidths,u_linewidths,cavity_lengths



In [ ]:
# V = 1.129
#8 um at  4.611, 
#so 
#removed 0.8+-0.03 @ N=22
#23,24,0.76,0.76,0.04,0.03,
fig,ax = plt.subplots()
ax.errorbar(cavity_lengths*1.e6,linewidths*1.e-9,yerr=u_linewidths*1.e-9,fmt='o')
ax.set_xlabel('cavity length (um)')
ax.set_ylabel('linewidth (GHz)')


In [ ]:
reload(common)
g_a = 0.1e9
g_b = (c)/(2*40000)
print g_b
print cavity_lengths,linewidths
p0, fitfunc, fitfunc_str = common.fit_inverse(g_a, g_b)
fit_result = fit.fit1d(cavity_lengths,linewidths, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=[])
print p0
fig,ax = plt.subplots(figsize=(8,4))
ax.plot(cavity_lengths,linewidths)
plot.plot_fit1d(fit_result, np.linspace(min(cavity_lengths),max(cavity_lengths),10*len(cavity_lengths)),ax=ax, label='Fit',show_guess=True, plot_data=True,color='red', data_linestyle = 'o', print_info= True)
b_fit=fit_result['params_dict']['b']
Finesse = c/(2*b_fit)
ax.set_title(str(Finesse))
# ax.text(0.1,0.1,'the finesse is '+str(Finesse))
print Finesse

In [ ]:
reload(oscillo_ana)
indir='D:\measuring\data/20160822/OND_POS2_L5_V9p832'#'D:\measuring\data/20160430\OFF_diamond'
filename='NICE_LWS00099'
EOM_freq=6. # in GHz

oa = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
oa.get_data(use_timetrace=False,x_min=0,x_max=1000)#1400)
oa.plot_data()#plot_mod=True)

# lws_old=lws[:-1]
lws = np.array([])#lws_old## 
nr_lws=120

x0 =860
windowsize=300# print len(x)
# xi = int(len(x)/nr_lws)
# print xi 

x,y = oa.get_data(use_timetrace=False)
# lw,u_lw = oa.plot_and_fit_with_EOM(EOM_freq=EOM_freq)
# print lw

for i in np.arange(nr_lws):
    xi = int(x0+(len(x)*i/nr_lws))
    
    if i%2==1:
        xi=xi-50   
    if (xi-windowsize)>0:
        x_min = xi-windowsize
    else: 
        x_min = 0
    if (xi+windowsize)<len(x)-1:
        x_max = xi+windowsize
    else:
        x_max =-1
    oa.get_data(use_timetrace=False,x_min = x_min, x_max=x_max)
    #oa.plot_data()
    lw,u_lw = oa.plot_and_fit_with_EOM(EOM_freq=EOM_freq,g_gamma1=20, g_dx=60,g_A2=2,g_A3=2,show_fit=True)
    lws = np.append(lws,lw)
#     if i>1:
#         break
        
print lws
print len(lws)
print np.average(lws)
print np.std(lws)

f = open(os.path.join(indir,'fitted_lws.txt'), 'w')
np.savetxt(f,lws)
f.close()

# plt.show()

# x,y = oscillo_ana.plot_data(indir, filename,use_timetrace=False,x_min=x_min,x_max=x_max)

# oscillo_ana.plot_and_fit_with_EOM(indir, filename,EOM_frq,g_a1, g_A1, g_x01, g_gamma1, g_dx, g_A2,g_A3,fixed=fixed,use_timetrace=False)

In [ ]:
# print len(lws)
# print lws
# lws_old=lws[120:]
# print len(lws_old)
# print lws_old
# #print lws_old
# f = open(os.path.join(indir,'fitted_lws.txt'), 'w')
# np.savetxt(f,lws_old)
# f.close()

In [ ]:
# indir='D:\measuring\data/20160819/OND_L2_V9p866'#

f = open(os.path.join(indir,'fitted_lws.txt'), 'r')
lws_from_file = np.loadtxt(f)
f.close()

print len(lws_from_file)

os.listdir(indir)

# filename='NICE_LWS00058'
reload(oscillo_ana)
oa = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
oa.get_data(use_timetrace=False,x_max=2000)#5000
oa.plot_data(plot_mod=True)
#oa.get_data(use_timetrace=False,x_min=-2000)
#oa.plot_data(plot_mod=True)

In [ ]:
#IF SINGLE FILE EXECUTE THIS CELL:

first_lw_direction = 'down'

updown_mean_lws,lws_up,lws_down =get_up_down_lws(lws_from_file,first_lw_direction)
analysis=add_to_analysis_dict({},lws_up,lws_down,updown_mean_lws)
save_to_json_file(indir,'analysis',analysis)

print 'results of ',indir
print_results(analysis)


In [ ]:
#IF THREE FILES EXECUTE THIS AND THE FOLLWING CELLS:

file1_first_lw_direction = 'up'
file2_first_lw_direction = 'up'
file3_first_lw_direction = 'down'
removedfromfile1=0
removedfromfile2=0


In [ ]:
fitted_lws_file1=lws_from_file[0:48-removedfromfile1]
fitted_lws_file2=lws_from_file[48-removedfromfile1:96-removedfromfile1-removedfromfile2]
fitted_lws_file3=lws_from_file[96-removedfromfile1-removedfromfile2:144]

print len(fitted_lws_file1),len(fitted_lws_file2),len(fitted_lws_file3)
#print fitted_lws_file1
#print fitted_lws_file2 
#print fitted_lws_file3

In [ ]:
save_to_txt_file(indir,'fitted_lws_file1',fitted_lws_file1)
save_to_txt_file(indir,'fitted_lws_file2',fitted_lws_file2)
save_to_txt_file(indir,'fitted_lws_file3',fitted_lws_file3)

updown_mean_lws_file1,lws_up_file1,lws_down_file1 =get_up_down_lws(fitted_lws_file1,file1_first_lw_direction)
updown_mean_lws_file2,lws_up_file2,lws_down_file2 =get_up_down_lws(fitted_lws_file2,file2_first_lw_direction)
updown_mean_lws_file3,lws_up_file3,lws_down_file3 =get_up_down_lws(fitted_lws_file3,file3_first_lw_direction)

updown_mean_lws = np.append(np.append(updown_mean_lws_file1,updown_mean_lws_file2),updown_mean_lws_file3)
lws_up = np.append(np.append(lws_up_file1,lws_up_file2),lws_up_file3)
lws_down = np.append(np.append(lws_down_file1,lws_down_file2),lws_down_file3)

analysis=add_to_analysis_dict({},lws_up,lws_down,updown_mean_lws)
analysis_file1=add_to_analysis_dict({},lws_up_file1,lws_down_file1,updown_mean_lws_file1)
analysis_file2=add_to_analysis_dict({},lws_up_file2,lws_down_file2,updown_mean_lws_file2)
analysis_file3=add_to_analysis_dict({},lws_up_file3,lws_down_file3,updown_mean_lws_file3)
save_to_json_file(indir,'analysis',analysis)
save_to_json_file(indir,'analysis_file1',analysis_file1)
save_to_json_file(indir,'analysis_file2',analysis_file2)
save_to_json_file(indir,'analysis_file3',analysis_file3)

print 'results of ',indir
print_results(analysis)
print_results(analysis_file1)
print_results(analysis_file2)
print_results(analysis_file3)

In [ ]:
import matplotlib.pyplot as plt

fig,ax=plt.subplots()

ax.plot(lws)
ax.set_xlabel('sweep nr')
ax.set_ylabel('linewidth')

In [ ]:
f = open(os.path.join(indir,'fitted_lws_file0010.txt'), 'w')
np.savetxt(f,lws)
f.close()

In [ ]:
copy_lws_from_file=lws_from_file
print co

In [ ]:
f = open(os.path.join(indir,'fitted_lws.txt'), 'w')
np.savetxt(f,new_lws_from_file)
f.close()